In [2]:
import pandas as pd
import plotly.express as px
from datetime import datetime
import pytz

# ---------------------------------
# Load dataset
# ---------------------------------
apps_df = pd.read_csv("googleplaystore.csv")

# ---------------------------------
# Data Cleaning
# ---------------------------------
apps_df['Installs'] = apps_df['Installs'].str.replace('[+,]', '', regex=True)
apps_df['Installs'] = pd.to_numeric(apps_df['Installs'], errors='coerce')

# ---------------------------------
# CREATE PSEUDO COUNTRY COLUMN (FIX)
# ---------------------------------
apps_df['Country'] = 'IND'   # Global installs mapped to India

# ---------------------------------
# Exclude categories starting with A, C, G, S
# ---------------------------------
apps_df = apps_df[
    ~apps_df['Category'].str.startswith(('A', 'C', 'G', 'S'), na=False)
]

# ---------------------------------
# Top 5 Categories by Installs
# ---------------------------------
top_categories = (
    apps_df.groupby('Category')['Installs']
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index
)

filtered_df = apps_df[apps_df['Category'].isin(top_categories)]

# ---------------------------------
# Aggregate installs by Country & Category
# ---------------------------------
map_df = filtered_df.groupby(
    ['Country', 'Category'], as_index=False
)['Installs'].sum()

# ---------------------------------
# Highlight installs > 1 million
# ---------------------------------
map_df['Highlight'] = map_df['Installs'].apply(
    lambda x: 'Above 1M' if x > 1_000_000 else 'Below 1M'
)

# ---------------------------------
# Time Restriction (6 PM – 8 PM IST)
# ---------------------------------
ist = pytz.timezone("Asia/Kolkata")
current_time = datetime.now(ist).time()

start_time = datetime.strptime("18:00", "%H:%M").time()
end_time = datetime.strptime("20:00", "%H:%M").time()

# ---------------------------------
# Conditional Dashboard Rendering
# ---------------------------------
if start_time <= current_time <= end_time:

    fig = px.choropleth(
        map_df,
        locations='Country',
        locationmode='ISO-3',
        color='Installs',
        hover_name='Country',
        hover_data=['Category', 'Installs', 'Highlight'],
        animation_frame='Category',
        color_continuous_scale='Plasma',
        title='Global App Installs by Category (Top 5)'
    )

    fig.update_layout(
        geo=dict(showframe=False, showcoastlines=True),
        coloraxis_colorbar=dict(title="Total Installs")
    )

    fig.show()

else:
    print("Choropleth map available only between 6:00 PM and 8:00 PM IST.")


Choropleth map available only between 6:00 PM and 8:00 PM IST.
